In [11]:
import pandas as pd
import numpy as np
import os
import swifter
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [12]:
user_metadata = pd.read_csv('models/user_metadata.tsv', sep='\t', header=None)
user_metadata.columns = ['account_id']
user_metadata.head()

,account_id
0,00029951-5ac0-477a-8e5a-6ecbb4131d61
1,00030859-3b2c-45c4-a957-a3b42c94891d
2,0003b04b-28fe-4ede-8123-c34ecf0ac370
3,0003e2b5-e7e4-4173-b901-9d39726a66e5
4,00049957-a93e-4aa5-be61-79b6826d07fc


In [13]:
user_metadata['embedding'] = pd.read_csv('models/user_embeddings.tsv', sep='\t', header=None).values.tolist()
user_metadata['embedding']

0          [0.22008125483989716, -0.5740570425987244, 0.0...
1          [0.35212284326553345, -0.2632900178432465, -0....
2          [0.0802295058965683, -0.05546204745769501, -0....
3          [0.8610720634460449, -0.2299892008304596, -0.1...
4          [0.50962632894516, -0.3746547698974608, -0.259...
5          [0.5985389351844788, -0.4775956869125367, -0.3...
6          [0.6739068031311035, -0.09017270058393478, -0....
7          [0.8328641057014465, -0.07072848081588745, 0.1...
8          [0.6572303175926208, 0.4838419556617737, -0.49...
9          [0.429103285074234, -0.01850120909512043, -0.2...
10         [-0.27748867869377136, 0.34312853217124933, 0....
11         [0.6808263659477234, 0.16070397198200226, -0.3...
12         [0.5875775814056396, -0.16629774868488312, -0....
13         [-0.04124073311686516, -0.018743926659226414, ...
14         [0.08438590914011002, 0.2947283089160919, 0.14...
15         [0.7221097946166991, 0.05625959113240242, 0.11...
16         [-0.039835244

In [14]:
user_metadata['embedding'][1]

[0.35212284326553345,
 -0.2632900178432465,
 -0.4542746245861053,
 0.1895538866519928,
 0.3669670522212983,
 -0.20481380820274356,
 0.07117751985788344,
 0.25834739208221436,
 -0.3463740646839142,
 -0.08890576660633087,
 0.2208274155855179,
 0.408968597650528,
 -0.1481698304414749,
 0.4762639999389648,
 0.19944310188293454,
 0.3855185508728027,
 -0.35873427987098694,
 0.07941659539937973,
 -0.20958368480205533,
 -0.26116582751274103,
 -0.3234803378582001,
 0.09286987036466597,
 0.1882687360048294,
 0.4533467888832092,
 -0.2937155365943909,
 0.5056282877922057,
 0.08057968318462372,
 0.17013272643089292,
 0.2465645372867584,
 0.0661911815404892,
 0.5892490148544312,
 -0.003736257553100586,
 0.3682863116264343,
 0.5861220955848694,
 -0.696108877658844,
 -0.060723248869180686,
 -0.5014252662658691,
 -0.3026226758956909,
 -0.20979230105876923,
 -0.0782397836446762,
 -0.28142833709716797,
 -0.52651309967041,
 0.11252475529909135,
 -0.5799943208694458,
 0.4475482404232025,
 0.043806452304124

In [15]:
restaurant_metadata = pd.read_csv('models/merchant_metadata.tsv', sep='\t', header=None)
restaurant_metadata.columns = ['merchant_id']
restaurant_metadata.head()

,merchant_id
0,175b424e-e990-49ed-be4d-70e5d54881cb
1,8037f763-9c92-441f-b77a-3f488ae072b5
2,899640d4-3c69-4826-b225-b1e3e86ef666
3,464835cb-f517-4b08-ba63-7b869d404afe
4,e8e671b4-b9db-462d-accc-8c5bf37fe155


In [16]:
restaurant_embeddings = pd.read_csv('models/restaurant_embeddings.tsv', sep='\t', header=None)
restaurant_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74
0,-0.872124,-1.276316,0.330462,0.859223,0.295120,0.951109,0.081153,0.769233,-0.949034,-0.464839,-0.402540,0.737685,0.711939,0.192739,-0.625988,-0.305814,-1.110032,0.655648,0.283790,-1.009441,-0.972292,-0.644182,0.874826,-0.682307,-0.101745,0.693527,-0.431450,0.017664,0.205152,-0.858716,0.419076,-1.187993,-0.179041,-0.677244,0.381837,0.664145,0.181284,-0.088388,0.564556,-0.037636,-0.947407,0.233806,1.143263,-1.269094,0.756095,-1.090349,0.256157,0.282577,0.824214,-0.770511,1.024808,1.113899,-0.591246,-0.156236,-0.519398,-1.159569,-0.141424,1.256564,-0.389516,-0.277443,0.205167,0.670219,-0.723227,-0.772914,0.184301,-0.331750,-1.734274,0.502862,0.521291,0.358579,-0.111788,-0.957612,-0.304952,-0.543854,-0.423050
1,-1.069470,0.093836,-0.500859,-0.190553,0.926890,-1.185735,-1.142283,0.776635,-0.744071,-0.320576,-0.337882,0.263383,-0.225308,0.809854,0.094945,1.204893,-0.428747,-0.084656,0.554341,-0.981585,-0.911163,-0.276575,-0.561496,1.449933,0.016663,0.151400,0.045508,0.925416,-0.494737,0.083648,0.474025,-0.088970,0.294168,0.972245,-0.311857,-0.508170,-0.615770,0.200793,0.122036,0.125939,0.052330,-0.955746,-1.075952,-0.611737,1.262013,-0.472199,-0.201158,0.201590,0.223323,-0.252005,-0.591395,-1.259205,-0.783648,0.410632,-0.655045,0.706250,-0.316212,0.015267,-1.472968,0.574607,0.605076,-1.074507,-0.616387,0.663757,-1.045597,0.442638,-0.064571,0.819687,-0.603703,0.868434,-0.374252,1.366890,1.241468,0.048523,0.012457
2,-0.135746,0.200677,0.067409,-0.320376,-0.334395,-0.255833,-0.310127,-0.806134,-0.089968,-0.191496,-1.334305,0.929539,0.649560,0.527981,1.188901,0.523787,0.430887,0.347876,1.449084,-0.745164,0.585980,0.589596,-0.190479,-0.673487,1.145455,0.024827,1.337513,-0.445422,0.733247,0.367723,1.315909,0.489835,-0.123838,0.243338,-1.005943,0.516530,-0.816961,0.050662,-0.389709,-0.928645,0.802396,0.358025,-1.501715,0.628899,-0.189042,0.353443,-0.792608,-0.677312,0.229383,0.447350,0.545078,0.765904,0.093522,0.017544,0.286238,-0.294356,0.059384,1.420387,-0.177560,-0.039352,0.002983,0.322759,0.577647,0.201168,0.981619,1.615226,1.292164,0.850829,-0.087134,-0.816011,-1.282616,-0.746738,-0.412553,0.397629,0.690462
3,-0.024711,0.344761,-0.403157,-0.397956,-0.700192,-1.065050,-0.352290,0.052252,-0.352912,-1.154379,-1.158853,-0.274598,0.766364,0.686270,0.024136,0.577166,-0.402114,0.576288,1.128805,-1.089535,0.694882,0.319456,-0.726233,0.167751,-0.076002,-0.721180,-0.039267,-0.324266,-0.785242,0.898325,0.620400,0.298818,0.672646,-0.334479,-0.235649,-0.486197,0.817447,-0.385137,-0.708263,-0.457744,1.084288,0.527640,-1.960067,-0.086138,0.470651,-0.824378,-0.714053,-0.859219,-0.558336,-0.435624,-0.453345,0.670710,0.266726,0.625942,-0.119384,-0.661769,0.444688,1.438630,0.374047,1.184984,-0.375694,1.075082,0.169224,0.469317,0.380600,1.030856,1.417506,0.442569,0.337858,-0.007654,-1.413455,0.792035,-0.465891,0.446779,0.507370
4,0.344261,-0.581673,-0.352533,-0.607911,-0.744296,1.274229,0.325355,-0.035959,-0.289983,-0.231977,1.199879,0.111037,1.067065,0.753345,-0.676412,-1.389755,0.454226,0.666350,-1.567583,-0.626015,-0.891668,-0.909908,0.139393,-0.396411,-0.380490,-0.653762,-0.863919,0.615254,-0.558576,0.719998,1.033687,-1.505563,0.986557,-0.300919,0.415698,0.480417,-0.234496,-0.800691,0.099386,-0.934219,0.221545,-0.612290,1.393192,-0.299127,-0.125086,0.210553,0.103848,0.315278,-0.351334,0.219465,1.720412,0.716333,-1.350008,-0.280095,0.528556,-0.602341,1.148187,0.931543,0.569744,0.614523,0.127698,-0.050630,0.560432,0.241387,-0.257532,0.571238,0.480716,-0.691738,0.836090,0.256718,0.958527,0.416208,0.433987,-0.215405,0.806291


In [17]:
restaurant_metadata['embedding'] = restaurant_embeddings.values.tolist()
restaurant_metadata.head()

,merchant_id,embedding
0,175b424e-e990-49ed-be4d-70e5d54881cb,"[-0.8721244931221007, -1.276315927505493, 0.33..."
1,8037f763-9c92-441f-b77a-3f488ae072b5,"[-1.069470167160034, 0.09383628517389295, -0.5..."
2,899640d4-3c69-4826-b225-b1e3e86ef666,"[-0.13574558496475217, 0.20067673921585086, 0...."
3,464835cb-f517-4b08-ba63-7b869d404afe,"[-0.024711422622203823, 0.3447608649730682, -0..."
4,e8e671b4-b9db-462d-accc-8c5bf37fe155,"[0.3442608714103699, -0.5816732048988342, -0.3..."


In [18]:
test_data_dir = '/opt/ifood_dataset/test_data'
file = 'part-00116-tid-1274133508071678345-c977427f-4769-4c15-80ee-9612d9a084d8-4793835-1-c000.snappy.parquet'
test_set = pd.read_parquet(os.path.join(test_data_dir, file))[['account_id', 'merchant_id', 'merc_list']]
test_set.head()

,account_id,merchant_id,merc_list
0,431dcc8c-6ea6-4717-8a41-dcf39bd7c2b1,debcd250-d1b8-4d9c-b200-af3b49888afc,"[ffa93591-dbfc-44a5-88a6-bfc2ff8edbd2, f2ff8c6..."
1,6dd03e9e-f1b1-4ea0-87b3-0fe361e96efb,bb8c5e3e-040b-428a-ab6c-11a6622b3dad,"[ff179724-6403-4588-9981-13a1deee8dc3, fe31dbf..."
2,2cadf5d4-f9c2-47bd-a821-cd6ea7b989a9,1daf9355-dd53-430e-97e7-a8c6aa77fd26,"[fefda585-9489-4ef2-96b0-3a7f4560a9b8, ebb1d02..."
3,43547609-b1e8-49ff-8512-78c41ebf1ec1,792d735f-e211-4c42-8fbf-ab25f866b3ae,"[fb5d2ebf-100b-4f43-af60-ddad284c19eb, f73f215..."
4,4a5a8aab-6db3-4d4d-a834-91f59e726977,a147a602-62dc-4584-bdbc-a676bacfa110,"[fe7a51d1-f116-4d0f-8810-0dae51458304, fd4a6b4..."


In [26]:
from math import log
import timeit

def dcg_at_k(scores):
    return scores[0] + sum(sc / log(ind, 2) for sc, ind in zip(scores[1:], range(2, len(scores)+1)))

def ndcg_at_k(scores):
    idcg = dcg_at_k(sorted(scores, reverse=True))
    return (dcg_at_k(scores) / idcg) if idcg > 0.0 else 0.0

def compute_cos_similarity(u, r, norm_user):
    if len(r) is not len(u):
        return 0.0
    return np.dot(u, r) / (np.linalg.norm(r) * norm_user)
    
def get_gains_from_ratings(ranks_ids, u, user_metadata, rests_metadata, merchant_code):
    merchant_index = rests_metadata.index.values[rests_metadata['merchant_id'] == merchant_code]
    if len(merchant_index) is 0:
        return [0]
    gains = (ranks_ids == merchant_index) * 1.0
    return gains

def compute_ndcg_at_k(u, user_metadata, rests_metadata):
#     start = timeit.default_timer()

    user_idx = u['account_id']
    user = user_metadata[user_metadata['account_id'].isin([user_idx])]
#     stop = timeit.default_timer()
#     print('Time Loc User: ', stop - start)  
#     start = stop
    if len(user) is 0:
        return 0.0
    user = np.array(user['embedding'].values[0])
    norm_user = np.linalg.norm(user)
    merc_list_embeddings = rests_metadata[rests_metadata['merchant_id'].isin(u['merc_list'])]
    #print(merc_list_embeddings)
    ranks = merc_list_embeddings.apply(lambda r: compute_cos_similarity(user, r['embedding'], norm_user), axis=1)
    #print(ranks)
    if len(ranks) is 0:
        return 0.0
    ranks = ranks.sort_values(ascending=False)
    ranks_ids = ranks.index.values.tolist()
    gains = get_gains_from_ratings(list(ranks_ids), user_idx, user_metadata, rests_metadata, u['merchant_id'])
    res = ndcg_at_k(gains)
    print(res)
    return res

#tqdm.pandas()
ndcg_users = test_set.swifter.progress_bar(enable=True).apply(lambda x: compute_ndcg_at_k(x, user_metadata, restaurant_metadata), axis=1)

0.0
0.5
0.16485085672216038
0.16604764621593782
0.20801459767650948
0.21533827903669653
0.27023815442731974
0.5
0.6309297535714574
0.2890648263178878
0.14332232775009315
0.1745834300480449
0.16260831227163416
0.6309297535714574
0.18920035951687003
0.0
0.19195872000656014
0.2890648263178878
0.38685280723454163
0.20801459767650948
1.0
0.5
0.21030991785715245
0.5
0.0
0.0
0.0
0.22106472945750374
0.0
0.0
0.0
0.5
0.43067655807339306
0.17905223175104135
0.3154648767857287
0.0
0.38685280723454163
0.1905514124267734
1.0
0.0
0.21274605355336312
0.0
0.15117821092177644
0.27023815442731974
0.227670248696953
1.0
0.1745834300480449
0.0
0.2559580248098155
0.13347832467375914
0.1255369169267456
0.3562071871080222
0.0
0.2890648263178878
0.13465719753215485
0.3333333333333333
0.20184908658209982
0.25
0.0
0.21533827903669653
0.18790182470910757
0.18428883314870617
0.0
0.0
1.0
0.3333333333333333
0.2559580248098155
0.227670248696953
1.0
0.43067655807339306
0.2789429456511298
0.0
0.43067655807339306
0.0
0.0

0.12974612741705907
0.30102999566398114
0.0
0.15643778834207156
0.1296303050907487
0.19195872000656014
0.26264953503719357
0.0
0.227670248696953
0.0
0.25
0.27023815442731974
0.0
1.0
0.19656163223282258
0.14924393652741208
0.23540891336663824
0.0
0.0
0.11759413125458612
1.0
0.1854490234153689
0.20184908658209982
0.3154648767857287
0.18665241123894338
0.18665241123894338
0.0
0.3333333333333333
0.0
1.0
0.0
0.3333333333333333
0.0
0.20801459767650948
0.0
0.0
0.24465054211822598
1.0
0.25
0.0
0.22424382421757544
0.38685280723454163
0.3562071871080222
0.2
0.30102999566398114
0.13391173292339814
0.1610447717564444
1.0
0.1089335406477727
0.26264953503719357
0.21533827903669653
0.6309297535714574
0.3333333333333333
0.21274605355336312
0.23540891336663824
0.27023815442731974
1.0
0.1762914343888821
0.0
0.0
0.21810429198553152
0.25
0.0
0.1729696904450771
0.16155546744299637
0.2559580248098155
0.15863495891559604
1.0
0.18003132665669264
0.23981246656813146
0.227670248696953
0.0
0.2789429456511298
0.0

0.0
0.5
0.16485085672216038
0.16604764621593782
0.20801459767650948
0.21533827903669653
0.27023815442731974
0.5
0.6309297535714574
0.2890648263178878
0.14332232775009315
0.1745834300480449
0.16260831227163416
0.6309297535714574
0.18920035951687003
0.0
0.19195872000656014
0.2890648263178878
0.38685280723454163
0.20801459767650948
1.0
0.5
0.21030991785715245
0.5
0.0
0.0
0.0
0.22106472945750374
0.0
0.0
0.0
0.5
0.43067655807339306
0.17905223175104135
0.3154648767857287
0.0
0.38685280723454163
0.1905514124267734
1.0
0.0
0.21274605355336312
0.0
0.15117821092177644
0.27023815442731974
0.227670248696953
1.0
0.1745834300480449
0.0
0.2559580248098155
0.13347832467375914
0.1255369169267456
0.3562071871080222
0.0
0.2890648263178878
0.13465719753215485
0.3333333333333333
0.20184908658209982
0.25
0.0
0.21533827903669653
0.18790182470910757
0.18428883314870617
0.0
0.0
1.0
0.3333333333333333
0.2559580248098155
0.227670248696953
1.0
0.43067655807339306
0.2789429456511298
0.0
0.43067655807339306
0.0
0.0

0.3333333333333333
0.0
0.27023815442731974
1.0
0.0
0.1515171524096389
0.0
0.27023815442731974
0.0
0.20584683246043442
0.2890648263178878
0.19342640361727081
0.43067655807339306
0.1572932473495469
0.0
0.0
0.2890648263178878
0.22424382421757544
1.0
0.22424382421757544
0.0
1.0
0.227670248696953
0.1905514124267734
0.0
0.38685280723454163
0.21274605355336312
0.0
0.14478295061395813
0.18428883314870617
0.1762914343888821
0.15643778834207156
0.27023815442731974
0.24465054211822598
0.23981246656813146
0.5
0.0
0.0
0.0
0.0
0.0
0.20584683246043442
0.1781035935540111
0.38685280723454163
0.20584683246043442
0.25
0.3562071871080222
1.0
0.0
0.15909880786929415
1.0
0.15863495891559604
0.43067655807339306
0.3562071871080222
0.21030991785715245
0.15863495891559604
0.2890648263178878
0.0
0.0
0.30102999566398114
0.3562071871080222
0.1754250635819545
0.0
0.0
0.20801459767650948
0.2559580248098155
0.38685280723454163
0.2559580248098155
0.14308844150498737
0.0
0.0
0.0
0.0
1.0
0.25
0.6309297535714574
0.301029

0.5
0.1440400303421672
0.12917448356450073
0.3154648767857287
0.1322359088617821
1.0
0.0
0.23137821315975915
0.15329048865267808
0.30102999566398114
0.2559580248098155
0.1754250635819545
1.0
0.0
0.5
0.16794877895704194
0.0
0.0
0.18665241123894338
0.0
0.19823986317056053
0.0
0.2890648263178878
0.22106472945750374
0.20184908658209982
0.0
0.2559580248098155
0.19342640361727081
0.16427204996205022
0.15366188628986424
0.5
0.0
0.21274605355336312
1.0
0.0
0.24465054211822598
0.1721954337940981
1.0
0.0
0.0
1.0
0.0
0.1552085627701551
0.38685280723454163
1.0
0.21274605355336312
0.38685280723454163
0.30102999566398114
0.5
0.15186115333086322
1.0
0.3154648767857287
0.0
0.1854490234153689
0.1762914343888821
0.3333333333333333
0.3333333333333333
0.18003132665669264
0.1686130986895011
0.17905223175104135
0.21810429198553152
1.0
0.19823986317056053
0.2890648263178878
0.1949590218937863
0.17144160057391344
1.0
1.0
0.18665241123894338
0.18316925091363362
0.0
0.6309297535714574
0.0
0.0
0.1522103467132433

...

1.0
0.0
0.6309297535714574
0.0
0.0
1.0
0.22106472945750374
0.22424382421757544
0.21533827903669653
1.0
0.16730017881017412
1.0
0.38685280723454163
1.0
0.0
1.0
0.3333333333333333
0.0
0.14717968691798522
0.16485085672216038
0.15863495891559604
0.13889520968509134
0.0
0.0
0.3154648767857287
0.17144160057391344
0.12482481998286134
0.0
0.21274605355336312
1.0
0.0
0.14924393652741208
0.21274605355336312
0.0
0.0
1.0
0.21810429198553152
0.38685280723454163
1.0
0.2559580248098155
1.0
0.18665241123894338
0.43067655807339306
0.0
0.21810429198553152
0.23540891336663824
0.0
0.1762914343888821
0.15084420062289414
0.0
0.14285714285714285
0.15957141566993815
1.0
0.23137821315975915
0.43067655807339306
0.0
0.18920035951687003
0.0
0.0
0.6309297535714574
0.2
0.26264953503719357
0.20801459767650948
0.16666666666666666
0.2789429456511298
1.0
0.3333333333333333
0.20584683246043442
0.0
1.0
0.1417377701235801
0.15084420062289414
0.38685280723454163
0.15084420062289414
0.2890648263178878
0.0
0.5
0.315464876785

0.5
0.5
0.27023815442731974
0.19656163223282258
0.38685280723454163
0.23137821315975915
0.17905223175104135
0.43067655807339306
0.12974612741705907
1.0
1.0
0.5
0.13511907721365987
0.0
0.3333333333333333
0.19342640361727081
0.1228568942814387
0.18790182470910757
0.21533827903669653
0.14987054163194743
0.1781035935540111
0.16666666666666666
0.2890648263178878
0.0
0.13465719753215485
0.16929380759878143
0.1529248683028321
0.16485085672216038
0.43067655807339306
0.30102999566398114
0.0
0.0
1.0
0.12917448356450073
0.0
0.25
0.0
0.38685280723454163
0.15957141566993815
0.6309297535714574
0.16485085672216038
0.0
0.2
0.27023815442731974
0.1515171524096389
0.0
0.0
0.26264953503719357
0.6309297535714574
0.5
0.38685280723454163
0.43067655807339306
0.2
0.125
0.0
0.22424382421757544
0.38685280723454163
0.6309297535714574
0.14006482349398794
0.0
0.0
0.38685280723454163
0.0
0.0
0.38685280723454163
0.3333333333333333
0.18316925091363362
0.22424382421757544
0.26264953503719357
0.0
0.30102999566398114
0.0

0.5
0.0
0.16207652439312228
0.12998007347308715
0.0
0.13852010756717745
0.23981246656813146
0.0
1.0
0.43067655807339306
0.19195872000656014
0.0
0.16604764621593782
0.0
0.0
0.0
1.0
0.0
0.22424382421757544
1.0
0.5
0.20584683246043442
0.1754250635819545
0.3562071871080222
0.0
0.17144160057391344
0.3562071871080222
0.0
0.6309297535714574
1.0
0.2
0.27023815442731974
0.0
0.1729696904450771
0.0
0.0
0.22106472945750374
0.38685280723454163
1.0
0.0
0.6309297535714574
0.18665241123894338
0.6309297535714574
0.20584683246043442
0.13480990596580794
0.14555268036201674
0.0
0.14987054163194743
1.0
0.0
0.43067655807339306
0.2789429456511298
0.2890648263178878
0.3562071871080222
0.0
0.0
0.27023815442731974
0.15117821092177644
1.0
0.16794877895704194
1.0
0.3333333333333333
0.23540891336663824
0.24465054211822598
0.21274605355336312
0.1854490234153689
0.0
0.38685280723454163
0.3154648767857287
0.3154648767857287
0.22424382421757544
0.2789429456511298
0.16427204996205022
0.0
0.0
1.0
1.0
0.5
0.6309297535714

1.0
0.1781035935540111
0.0
0.16730017881017412
0.15442266280111014
0.22106472945750374
0.43067655807339306
0.0
0.3154648767857287
0.3562071871080222
0.0
0.14635095807586196
0.17144160057391344
0.3562071871080222
0.24465054211822598
0.18003132665669264
0.0
0.1949590218937863
0.21274605355336312
0.2559580248098155
0.5
0.22106472945750374
0.2
0.23981246656813146
0.43067655807339306
0.16544255391905832
1.0
0.0
0.20584683246043442
0.19823986317056053
0.26264953503719357
0.0
0.17376534287144
0.1529248683028321
0.0
0.0
0.23137821315975915
0.38685280723454163
0.15773243839286435
1.0
0.2890648263178878
0.27023815442731974
0.0
0.0
0.3154648767857287
0.43067655807339306
0.6309297535714574
0.26264953503719357
0.19195872000656014
1.0
0.23981246656813146
0.18428883314870617
0.38685280723454163
0.18665241123894338
0.26264953503719357
0.14581551052860536
0.0
0.3154648767857287
1.0
0.0
0.6309297535714574
0.20584683246043442
0.0
0.1729696904450771
0.11608052019106711
0.1721954337940981
0.165442553919058

0.0
0.0
0.0
0.0
0.16666666666666666
0.2559580248098155
0.27023815442731974
0.0
1.0
0.0
0.0
0.23540891336663824
0.30102999566398114
0.21533827903669653
0.21810429198553152
0.0
0.0
0.43067655807339306
1.0
0.27023815442731974
0.21810429198553152
0.23981246656813146
0.5
0.22424382421757544
0.17905223175104135
0.25
0.156021065022225
0.25
0.19823986317056053
0.27023815442731974
1.0
0.1319708865228925
1.0
0.3562071871080222
0.5
0.21533827903669653
0.21030991785715245
0.6309297535714574
1.0
0.1729696904450771
0.26264953503719357
0.15084420062289414
0.0
0.0
0.19823986317056053
0.15084420062289414
0.5
0.18920035951687003
0.0
0.13347832467375914
0.1581795909397823
1.0
0.21810429198553152
0.18790182470910757
0.25
0.0
0.0
0.16730017881017412
0.2890648263178878
0.13691539142239212
0.15909880786929415
0.2789429456511298
0.0
0.3333333333333333
0.0
0.0
0.24465054211822598
0.1610447717564444
0.15863495891559604
0.0
0.2890648263178878
0.21533827903669653
0.3562071871080222
0.21533827903669653
0.185449023

In [ ]:
ndcg_users_without_zero = ndcg_users[ndcg_users > 0.0]
print(ndcg_users_without_zero.shape)
print(ndcg_users_without_zero.mean(), ndcg_users_without_zero.std())
ndcg_users_without_zero.hist(bins=50, density=True)
plt.show()

In [11]:
import random
def compute_ndcg_random_list(u):
    x = u['merc_list']
    random.shuffle(x)
    ranks = list(map(lambda k: (k == u['merchant_id']) * 1.0, x))
    print(ranks)
    ndcg = ndcg_at_k(ranks)
    print(ndcg)
    return ndcg
    
    

random_ndcg = test_set.apply(lambda x: compute_ndcg_random_list(x), axis=1)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.30102999566398114
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.23137821315975915
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
0.

[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.6309297535714574
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
0.17144160057391344
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.23137821315975915
[1.0, 0.0]
1.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

...

0.0
[0.0]
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
0.23540891336663824
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
0.23137821315975915
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.25
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

KeyboardInterrupt: 

In [ ]:
random_ndcg_without_zero = random_ndcg[random_ndcg > 0.0]
print(random_ndcg_without_zero.shape)
print(random_ndcg_without_zero.mean(), random_ndcg_without_zero.std())
random_ndcg_without_zero.hist(bins=50, density=True)
plt.show()